In [9]:
import numpy as np

In [10]:
def downsample(image, pool_size, stride, pooling_function):
  """
  Generic downsampling function.
  image: 2D numpy array (grayscale)
  pool_size: tuple (ph, pw)
  stride: tuple (sh, sw)
  pooling_function: function(block) -> pixel value
  """

  # Compute output size based on passed parameters
  in_height, in_width = image.shape
  out_height = (in_height - pool_size[0]) // stride[0] + 1
  out_width  = (in_width  - pool_size[1]) // stride[1] + 1

  # Construct empty output image
  output = np.zeros((out_height, out_width), dtype=image.dtype)

  # Iterate over every pixel in the output image
  for y in range(out_height):
    for x in range(out_width):

      # Compute the corresponding x section of the input image
      x_start = x * stride[1]
      x_end = x * stride[1] + pool_size[1]

      # Compute the corresponding y section of the input image
      y_start = y * stride[0]
      y_end = y * stride[0] + pool_size[0]

      # Construct the corresponding source block and compute the maximum value
      block = image[y_start:y_end, x_start:x_end]
      output[y,x] = pooling_function(block)

  return output

In [11]:
def max_pooling_downsample(image, pool_size=(2,2), stride=(2,2)):
  """
  Max pooling function.
  image: 2D numpy array (grayscale)
  pool_size: tuple (ph, pw)
  stride: tuple (sh, sw)
  """

  # Define the max pooling function
  def max_pooling(block):
    return np.max(block)

  # Run downsampling with max pooling
  output = downsample(image, pool_size, stride, max_pooling)
  return output



def avg_pooling_downsample(image, pool_size=(2,2), stride=(2,2)):
  """
  Average pooling function.
  image: 2D numpy array (grayscale)
  pool_size: tuple (ph, pw)
  stride: tuple (sh, sw)
  """

  # Define the avg pooling function
  def avg_pooling(block):
    return np.mean(block)

  # Run downsampling with avg pooling
  output = downsample(image, pool_size, stride, avg_pooling)
  return output


def med_pooling_downsample(image, pool_size=(2,2), stride=(2,2)):
  """
  Median pooling function.
  image: 2D numpy array (grayscale)
  pool_size: tuple (ph, pw)
  stride: tuple (sh, sw)
  """

  # Define the med pooling function
  def med_pooling(block):
    return np.median(block)

  # Run downsampling with med pooling
  output = downsample(image, pool_size, stride, med_pooling)
  return output


In [12]:
def upsample(image, scale, interpolation_function):
  """
  Generic upsampling function.
  image: 2D numpy array (grayscale)
  scale: tuple (sh, sw)
  interpolation_function: function(image, src_y, src_x) -> pixel value
  """

  # Compute output image size
  in_height, in_width = image.shape
  out_height = in_height * scale[0]
  out_width  = in_width * scale[1]

  # Construct output image
  output = np.zeros((out_height, out_width), dtype=image.dtype)

  # Iterate over every pixel of the output image
  for y in range(out_height):
    for x in range(out_width):
      src_y = y / scale[0]
      src_x = x / scale[1]

      # Check if this pixel maps back to one of the original pixels
      if (src_y % 1 == 0 and src_x % 1 == 0):
        output[y, x] = image[int(src_y), int(src_x)]

      # If not, use interpolation to determine pixel value
      else:
        output[y, x] = interpolation_function(image, src_y, src_x)

  return output

In [13]:
def nn_interpolation(image, src_y, src_x):
  """
  Nearest neighbour interpolation function.
  image: 2D numpy array (grayscale)
  src_x: float
  src_y: float
  """

  # Round to nearest integer coordinates
  closest_y = int(round(src_y))
  closest_x = int(round(src_x))

  # Ensure nearest integer coordinates are within image bounds
  closest_y = min(max(closest_y, 0), image.shape[0] - 1)
  closest_x = min(max(closest_x, 0), image.shape[1] - 1)

  return image[closest_y, closest_x]


def bilinear_interpolation(image, src_y, src_x):
  """
  Bilinear interpolation function using averaging.
  image: 2D numpy array (grayscale)
  src_x: float
  src_y: float
  """

  # Find surrounding integer coordinates
  y_min = int(np.floor(src_y))
  y_max = int(np.ceil(src_y))
  x_min = int(np.floor(src_x))
  x_max = int(np.ceil(src_x))

  # Ensure surrounding integer coordinates are within image bounds
  y_min = max(y_min, 0)
  y_max = min(y_max, image.shape[0] - 1)
  x_min = max(x_min, 0)
  x_max = min(x_max, image.shape[1] - 1)

  # Construct surrounding integer coordinates
  top_left = image[y_min, x_min]
  top_right = image[y_min, x_max]
  bottom_left = image[y_max, x_min]
  bottom_right = image[y_max, x_max]

  # Compute distances
  dy = src_y - y_min
  dx = src_x - x_min

  # Compute average pixel value
  weighted_average_value = (1 - dy) * (1 - dx) * top_left + \
                  (1 - dy) * dx * top_right + \
                  dy * (1 - dx) * bottom_left + \
                  dy * dx * bottom_right

  return weighted_average_value


def bicubic_interpolation(image, src_y, src_x):
  """
  Two-layered bicubic interpolation function using averaging.
  image: 2D numpy array (grayscale)
  src_x: float
  src_y: float
  """

  # Define layer weights and values
  inner_layer_weight = 0.6
  outer_layer_weight = 0.4

  # Find the pixel to the top left in the original image
  y0 = int(np.floor(src_y))
  x0 = int(np.floor(src_x))

  # Walk a 4x4 grid around the (src_x, src_y) in the original image
  weighted_values = []
  for m in range(-1, 3):
    for n in range (-1, 3):
      y = y0 + m
      x = x0 + n

      # Ignore values outside the bounds of the image
      if (y < 0 or y > image.shape[0] - 1 or x < 0 or x > image.shape[1] - 1):
        continue

      # Handle outer layer points
      if m == -1 or m == 2 or n == -1 or n == 2:
        weighted_values.append(image[y, x] * outer_layer_weight)

      # Handle inner layer points
      else:
        weighted_values.append(image[y, x] * inner_layer_weight)

  weighted_average_value = sum(weighted_values) / len(weighted_values)

  return weighted_average_value


In [14]:
test_img = np.array([
    [ 10,  20,  30,  40,  50,  60,  70,  80,  90, 100],
    [ 15,  25,  35,  45,  55,  65,  75,  85,  95, 105],
    [ 20,  30,  40,  50,  60,  70,  80,  90, 100, 110],
    [ 25,  35,  45,  55,  65,  75,  85,  95, 105, 115],
    [ 30,  40,  50,  60,  70,  80,  90, 100, 110, 120],
    [ 35,  45,  55,  65,  75,  85,  95, 105, 115, 125],
    [ 40,  50,  60,  70,  80,  90, 100, 110, 120, 130],
    [ 45,  55,  65,  75,  85,  95, 105, 115, 125, 135],
    [ 50,  60,  70,  80,  90, 100, 110, 120, 130, 140],
    [ 55,  65,  75,  85,  95, 105, 115, 125, 135, 145]
], dtype=np.uint8)

In [15]:
def test_max_pooling():
    print("Test 1: Max Pooling Downsample (2x2, stride 2)")
    out = max_pooling_downsample(test_img, pool_size=(2,2), stride=(2,2))
    print(out, "\n")

def test_avg_pooling():
    print("Test 2: Average Pooling Downsample (2x2, stride 2)")
    out = avg_pooling_downsample(test_img, pool_size=(2,2), stride=(2,2))
    print(out, "\n")

def test_med_pooling():
    print("Test 3: Median Pooling Downsample (2x2, stride 2)")
    out = med_pooling_downsample(test_img, pool_size=(2,2), stride=(2,2))
    print(out, "\n")

def test_nn_upsample():
    print("Test 4: Nearest Neighbor Upsample (scale=2)")
    out = upsample(test_img, (2,2), nn_interpolation)
    print(out, "\n")

def test_bilinear_upsample():
    print("Test 5: Bilinear Upsample (scale=2)")
    out = upsample(test_img, (2,2), bilinear_interpolation)
    print(out, "\n")

def test_bicubic_upsample():
    print("Test 6: Bicubic Upsample (scale=2)")
    out = upsample(test_img, (2,2), bicubic_interpolation)
    print(out, "\n")


test_max_pooling()
test_avg_pooling()
test_med_pooling()
test_nn_upsample()
test_bilinear_upsample()
test_bicubic_upsample()

Test 1: Max Pooling Downsample (2x2, stride 2)
[[ 25  45  65  85 105]
 [ 35  55  75  95 115]
 [ 45  65  85 105 125]
 [ 55  75  95 115 135]
 [ 65  85 105 125 145]] 

Test 2: Average Pooling Downsample (2x2, stride 2)
[[ 17  37  57  77  97]
 [ 27  47  67  87 107]
 [ 37  57  77  97 117]
 [ 47  67  87 107 127]
 [ 57  77  97 117 137]] 

Test 3: Median Pooling Downsample (2x2, stride 2)
[[ 17  37  57  77  97]
 [ 27  47  67  87 107]
 [ 37  57  77  97 117]
 [ 47  67  87 107 127]
 [ 57  77  97 117 137]] 

Test 4: Nearest Neighbor Upsample (scale=2)
[[ 10  10  20  30  30  30  40  50  50  50  60  70  70  70  80  90  90  90
  100 100]
 [ 10  10  20  30  30  30  40  50  50  50  60  70  70  70  80  90  90  90
  100 100]
 [ 15  15  25  35  35  35  45  55  55  55  65  75  75  75  85  95  95  95
  105 105]
 [ 20  20  30  40  40  40  50  60  60  60  70  80  80  80  90 100 100 100
  110 110]
 [ 20  20  30  40  40  40  50  60  60  60  70  80  80  80  90 100 100 100
  110 110]
 [ 20  20  30  40  40  40  50